In [8]:
import torch
import torch.nn as nn

In [18]:
from collections import OrderedDict


In [25]:
class conv1dNet(nn.Module):
    def __init__(self, 
                 in_features,
                 kernel_size=7,
                 stride=2,
                 kernel_size_Pool1d=7,
                 num_layers = 5,
                 multiplier_of_channel=2,
                 dropout=0.01,
                 seed=None):
        
        """
        Модель собственной разработки
        """
        super().__init__()
#         set_determenistic(seed)
        in_channels = 1 
        out_channels = multiplier_of_channel*in_channels
        
        
        def single1dblock(in_channels,
                          out_channels,
                          kernel_size,
                          kernel_size_Pool1d,
                          stride):
            """
            Конструкция для 1 признака для 1 слоя
            """
            one_block = nn.Sequential(\
                                      OrderedDict([\
                                                   ('conv', nn.Conv1d(in_channels,
                                                                      out_channels,
                                                                      kernel_size,
                                                                      stride=stride,
                                                                      padding=0)),
                                                   ('pool', nn.MaxPool1d(kernel_size_Pool1d,
                                                                         stride=stride,
                                                                         padding=0)),
                                                   ('drop', nn.Dropout(p=dropout)),
                                                   ('relu', nn.ReLU())
                                                  ])\
                                     )
            return one_block


        

        def layers_for_1feat(in_channels,
                             out_channels,
                             kernel_size,
                             kernel_size_Pool1d,
                             multiplier_of_channel,
                             num_layers,
                             stride):
            """
            Конструкция слоев для 1 признака
            """            
            layers = []
            for i in range(num_layers):
                layers.append(\
                              (f'block{i}',
                               single1dblock(in_channels = in_channels,
                                             out_channels = out_channels,
                                             kernel_size = kernel_size,
                                             kernel_size_Pool1d = kernel_size_Pool1d,
                                             stride = stride)
                              )
                             )
                in_channels = out_channels
                out_channels = out_channels * multiplier_of_channel 
            layers = nn.Sequential(OrderedDict(layers))
            return layers

        """
        Здесь начинается реализация заданных блоков
        """
        main_body = []
        for i in range(in_features):
            main_body.append(layers_for_1feat(\
                                              in_channels = in_channels,
                                              out_channels = out_channels,
                                              kernel_size = kernel_size,
                                              kernel_size_Pool1d = kernel_size_Pool1d,
                                              multiplier_of_channel = multiplier_of_channel,
                                              num_layers = num_layers,
                                              stride = stride,
                                             )
                            )
        main_body = nn.ModuleList(main_body)
        self.main_body = main_body
        self.drop = nn.Dropout(p=dropout)
        self.linear = None
        
        x = torch.ones((1,(kernel_size+kernel_size_Pool1d)*stride*num_layers*10,in_channels)).float()
        print(x.shape)
        y = self.forward(x)
        print(y.shape)

        
    def run_epoch(self,
                  iterator,
                  optimizer,
                  criterion,
                  points_ahead=1,
                  phase='train', 
                  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
        
        self.to(device)
        is_train = (phase == 'train')
        if is_train:
            self.train()
        else:
            self.eval()
        
        epoch_loss = 0
        all_y_preds = []
        with torch.set_grad_enabled(is_train):
            for i, (x,y) in enumerate(iterator):
                x,y = np.array(x),np.array(y) 
                
                x = torch.tensor(x).float().to(device).requires_grad_()
                y_true = torch.tensor(y).float().to(device)
                y_pred = self.forward(x,device)

                if phase == 'forecast':
                    all_y_preds.append(y_pred)
                    continue 
                loss = criterion(y_pred,y_true)  
                if is_train:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                epoch_loss += loss.item()
        
        if phase != 'forecast':
            return epoch_loss / len(iterator)
        else:
            return torch.cat(all_y_preds).detach().cpu().numpy()
    
    
    def forward(self, x):

        y_common = []              
        for i in range(x.shape[2]):
            y_common.append(self.main_body[i](x[:,:,i].unsqueeze(1)))
        y_common = torch.cat(y_common,axis=1).view((x.shape[0],-1))
        
#         if self.linear is None:
#             self.linear = nn.Linear(y_common.size(1), 1)
#             self.linear.to(device)
#             print(self.linear)
            

        
#         y_common = self.linear(self.drop(y_common))
        return y_common
    


In [26]:
model = conv1dNet(10)

torch.Size([1, 1400, 1])


RuntimeError: Given input size: (16x1x5). Calculated output size: (16x1x0). Output size is too small